In [1]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import json
# Load the pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
#
model = BertModel.from_pretrained('bert-base-chinese')
# Define the corpus of documents


c:\Users\Dan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 110k/110k [00:00<00:00, 1.07MB/s]
c:\Users\Dan\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mod

[3 2 1 0]
Document:  荷蘭隊會在今年經典賽大敗給中華隊
Cosine similarity score:  0.9976505

Document:  統一獅會是今年中華職棒總冠軍，我超級喜歡看球賽
Cosine similarity score:  0.9968725

Document:  免疫學拓荒者韓韶華追思會 醫界齊聚緬懷共同導師
Cosine similarity score:  0.9958317

Document:  稱俄烏戰爭「是針對我們發動的」 俄外長發言引哄堂大笑
Cosine similarity score:  0.9941991



## define corpus

In [ ]:
corpus = ["稱俄烏戰爭「是針對我們發動的」 俄外長發言引哄堂大笑", "免疫學拓荒者韓韶華追思會 醫界齊聚緬懷共同導師", "統一獅會是今年中華職棒總冠軍，我超級喜歡看球賽", "荷蘭隊會在今年經典賽大敗給中華隊"]

In [ ]:

# Tokenize and encode the documents
max_length = 512
token_ids_list = [tokenizer.encode(doc, add_special_tokens=True, max_length=max_length, pad_to_max_length=True) for doc in corpus]
tokens_tensor = torch.tensor(token_ids_list)

# Get the BERT model output for the token IDs
with torch.no_grad():
    output = model(tokens_tensor)

# Get the last-layer hidden states from the model output
last_hidden_state = output.last_hidden_state

# Compute document embeddings by mean pooling the word embeddings
document_embeddings = torch.mean(last_hidden_state, dim=1)



In [2]:
document_embeddings

tensor([[ 0.8316,  0.6865, -0.3569,  ...,  1.3948, -0.1248,  0.6666],
        [ 0.8504,  0.6696, -0.4067,  ...,  1.4203, -0.1244,  0.6608],
        [ 0.8850,  0.6766, -0.3495,  ...,  1.3402, -0.1016,  0.6599],
        [ 0.8943,  0.6613, -0.3665,  ...,  1.3540, -0.1030,  0.6779]])

In [ ]:
# Define the query document
query_document = '我超級喜歡看球賽'

# Encode the query document
query_token_ids = tokenizer.encode(query_document, add_special_tokens=True, max_length=max_length, pad_to_max_length=True)
query_tokens_tensor = torch.tensor([query_token_ids])

# Get the BERT model output for the query document
with torch.no_grad():
    query_output = model(query_tokens_tensor)

# Get the last-layer hidden states from the model output for the query document
query_last_hidden_state = query_output.last_hidden_state

# Compute the query document embedding by mean pooling the word embeddings
query_document_embedding = torch.mean(query_last_hidden_state, dim=1)



In [3]:
query_document_embedding

tensor([[ 1.0193e+00,  6.8079e-01, -4.2355e-01,  1.2025e-01,  1.2165e+00,
          2.1201e-01,  5.4072e-02,  1.8587e-01, -1.1844e-01, -1.4872e-01,
          4.9715e-03,  1.8081e-01, -1.9555e-01, -8.7024e-01, -5.7365e-01,
          8.9012e-01,  4.9895e-01, -5.3013e-01,  4.5205e-02, -4.8825e-01,
         -2.1294e-01, -1.5432e+00, -3.3531e-01,  1.2212e-01,  6.5850e-01,
         -3.7862e-01,  6.7309e-01, -9.1148e-01, -2.0345e+00,  4.8382e-01,
          3.4822e-01,  1.2345e+00, -8.7425e-01, -1.2021e+00, -8.6886e-01,
          2.3102e-01, -1.8267e-01, -3.5394e-01, -8.0446e-02,  2.3358e-01,
          8.8538e-01, -6.8714e-01, -1.3028e-01, -1.4613e+00,  1.2544e-01,
          8.3212e-01,  7.5333e-01,  6.1635e-01,  6.3930e-01,  9.9930e-01,
         -7.4460e-01,  7.9773e+00, -3.5110e-01,  2.1924e-01, -7.3828e-01,
         -1.3501e-01,  1.3776e+00, -3.1929e-01,  6.1334e-01, -4.9405e-02,
          3.2238e-02,  4.2151e-01, -2.5599e-01, -3.6155e-01,  1.8321e-02,
          8.3527e-01,  2.3358e-01, -6.

In [ ]:
# Compute the pairwise cosine similarity between the query document embedding and all document embeddings
# [:,None]  --->  [1,2,3]->[[1],[2],[3]]
# q.d / |q||d|
query_cosine_similarities = np.dot(query_document_embedding, document_embeddings.T) / (np.linalg.norm(query_document_embedding, axis=1)[:, None] * np.linalg.norm(document_embeddings, axis=1))



In [ ]:
# Find the top-k most similar documents to the query document
# [::-1][:k]  -> [大到小][前k個]
k = 5
top_k_indices = np.argsort(query_cosine_similarities[0])[::-1][:k]
print(top_k_indices)
# Print the top-k most similar documents and their cosine similarity scores
for i in top_k_indices:
    print('Document: ', corpus[i])
    print('Cosine similarity score: ', query_cosine_similarities[0][i])
    print()